In [65]:
import pandas as pd
import nltk
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
import nltk.data
import collections

In [66]:
filenames = ["data/reviews_Home_and_Kitchen_5.json"]
filename = "data/reviews_Home_and_Kitchen_5.json"
file_name = "rawdata.pkl"
adversarial_file_name = "adversialdata.pkl"
max_features = 20000
maxlen = 100
batch_size = 32
num_class = 2

In [67]:
data = pd.read_pickle(file_name)

In [68]:
data.head(10)

,overall,reviewText
489257,1,I use this set for both general food storage i...
318868,0,Not a huge fan. I like to have a side for damp...
527578,1,I have always wanted to have a French press at...
102291,0,I purchased this to replace a breadmaker that ...
137178,0,"I bought this pan because my 12"" pan needed to..."
233040,0,"Just sayin'. I mean, honestly. Do we really ne..."
413128,1,"This rack fits the sink perfectly, and isn't b..."
149988,0,I bought one of the 20X72 inch mats for my wif...
18029,1,I have four different sizes of these scoops an...
129849,1,These replacement rubbers are perfect for use ...


### edit datafram

In [73]:
POS_TO_WORDNET = {
    'NN': wn.NOUN
#     'JJ': wn.ADJ,
#     'JJR': wn.ADJ,
#     'JJS': wn.ADJ,
}

In [74]:
def alter_wordnet_antonyms(token):
    if token[1] not in POS_TO_WORDNET: return None
    w = token[0].lower()
    wn_pos = POS_TO_WORDNET[token[1]]
    synsets = wn.synsets(w, wn_pos)
    if not synsets: return None
    synset = synsets[0]
    antonyms = []
    for lem in synset.lemmas():
        if lem.antonyms():
            for a in lem.antonyms():
                new_word = a.name()
                if '_' in a.name(): continue
                antonyms.append(new_word)
    return antonyms

In [75]:
texts = data['reviewText']
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [77]:
index = 0
for i, row in data.iterrows():
    text = texts[i]
    tokenized = tokenizer.tokenize(text)
    words = word_tokenize(text)
    tokens = nltk.pos_tag(words)
    output = ""
    for token in tokens:
        if token == '.':
            output = output + token
            index = 0
            break
        if index != 0:
            output = output + ' '
        antonyms = alter_wordnet_antonyms(token)
        if antonyms is None or len(antonyms) == 0:
            output = output + token[0]
        else:
#             print(i, ": change from ", token[0], " to ", antonyms[0])
            output = output + antonyms[0]
        index = index + 1
    data.at[i,'reviewText'] = output

In [ ]:
data.to_pickle(adversarial_file_name)

In [64]:
data['reviewText'].iloc[0]

"Iusethissetforbothgeneralfoodstorageinthefridgeaswellastakinglunchestowork.Ifwestillwentonregularcampingtripsthese(alongwithothercontainersfromLunchBots)wouldbestapleitemsinmybackpackastheyareverylight,easytoclean,andleakproofwhichisgreatontreks.Forstoringleftovers,IlovethatIcanseethroughthelids--asaslazyasitsoundswetendtoforgetaboutleftoverswhenwecan'tseewhat'sinthecontainerandthenfoodtendstogetwasted.Thesekeepthatfromhappening.Forlunchesormealsonthego,theseareincrediblylightandtotallyleak-proof.Iusuallyusethedeeperoneforalargesaladandtheshallowonefordinnerleftoverstohavewiththesalad.Thelidssnapontightyetareeasytoopenandclose.Agreatadditiontoanyhousehold."

In [231]:
for i in range(5):
    print(i, ": ")
    print(texts.iloc[i])
    print()

0 : 
I use this set for both general food storage in the fridge as well as taking lunches to work. If we still went on regular camping trips these (along with other containers from LunchBots) would be staple items in my backpack as they are very light, easy to clean, and leakproof which is great on treks.For storing leftovers, I love that I can see through the lids -- as as lazy as it sounds we tend to forget about leftovers when we can't see what's in the container and then food tends to get wasted. These keep that from happening.For lunches or meals on the go, these are incredibly light and totally leak-proof. I usually use the deeper one for a large salad and the shallow one for dinner leftovers to have with the salad.The lids snap on tight yet are easy to open and close.A great addition to any household.

1 : 
Not a huge fan. I like to have a side for damp clothes (towels and such after shower) and dry on the other. That would of been great had it not got moldy on the bottom after 

" I use this set for both general food storage in the fridge as well as taking lunches to work . If we still went on regular camping trips these ( along with other containers from LunchBots ) would be staple items in my backpack as they are very light , easy to clean , and leakproof which is great on treks.For storing leftovers , I love that I can see through the lids -- as as lazy as it sounds we tend to forget about leftovers when we ca n't see what 's in the container and then food tends to get wasted . These keep that from happening.For lunches or meals on the go , these are incredibly light and totally leak-proof . I usually use the deeper one for a large salad and the shallow one for dinner leftovers to have with the salad.The lids snap on tight yet are easy to open and close.A great addition to any household ."

In [72]:
wn.synsets('dog') # doctest: +ELLIPSIS +NORMALIZE_WHITESPACE

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01'),
 Synset('chase.v.01')]

In [73]:
wn.synsets('dog', pos=wn.VERB)

[Synset('chase.v.01')]

In [74]:
wn.synset('dog.n.01')

Synset('dog.n.01')

In [75]:
print(wn.synset('dog.n.01').definition())

a member of the genus Canis (probably descended from the common wolf) that has been domesticated by man since prehistoric times; occurs in many breeds


In [76]:
len(wn.synset('dog.n.01').examples())

1

In [77]:
print(wn.synset('dog.n.01').examples()[0])

the dog barked all night


In [78]:
wn.synset('dog.n.01').lemmas()

[Lemma('dog.n.01.dog'),
 Lemma('dog.n.01.domestic_dog'),
 Lemma('dog.n.01.Canis_familiaris')]

In [79]:
[str(lemma.name()) for lemma in wn.synset('dog.n.01').lemmas()]

['dog', 'domestic_dog', 'Canis_familiaris']

In [80]:
wn.lemma('dog.n.01.dog').synset()

Synset('dog.n.01')

In [85]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\vwslz\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True